<a href="https://colab.research.google.com/github/salo1802/AED/blob/main/TallerPipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Based on https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LinearRegression

In [66]:
X1 = [3,4,2,5,2,4,1,7]
X2 = [6,33,46,21,65,33,15,11]
y = y = [x + 3*(z/(z-1)) for x, z in zip(X1, X2)]



df = pd.DataFrame(columns=['X1', 'X2', 'y'], data=[
                                                   [X1[0],X2[0],y[0]],
                                                   [X1[1],X2[1],y[1]],
                                                   [X1[2],X2[2],y[2]],
                                                   [X1[3],X2[3],y[3]],
                                                   [X1[4],X2[4],y[4]],
                                                   [X1[5],X2[5],y[5]],
                                                   [X1[6],X2[6],y[6]],
                                                   [X1[7],X2[7],y[7]]
])

print(df)

### y = X1 + 2 * sqrt(X2)
# Esto asegura que un modelo de regresión lineal simple no pueda ajustarse perfectamente.

   X1  X2          y
0   3   6   6.600000
1   4  33   7.093750
2   2  46   5.066667
3   5  21   8.150000
4   2  65   5.046875
5   4  33   7.093750
6   1  15   4.214286
7   7  11  10.300000


In [67]:
# Separamos en train y test

train = df.iloc[:6]
test = df.iloc[6:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [68]:
# Creamos una clase transformadora


class ExperimentalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    print('\n>>>>>>>init() called.\n')

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() # creamos una copia para evitar cambios en el conjunto de datos original
    X_.X2 = 3 * (X_.X2/(X_.X2-1))
    return X_

In [70]:
# sin transformación de entrada


print("create pipeline 1")
pipe1 = Pipeline(steps=[
                       ('linear_model', LinearRegression())
])

print("fit pipeline 1")
pipe1.fit(train_X, train_y)

print("predict via pipeline 1")
preds1 = pipe1.predict(test_X)

print(f"\n{preds1}")  #
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds1))}\n")

create pipeline 1
fit pipeline 1
predict via pipeline 1

[ 4.66628785 10.03711863]
RMSE: 0.36973811397208417



In [74]:
# con transformation de entradas

print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer()),
                       ('linear_model', LinearRegression())
])





print("fit pipeline 2")
pipe2.fit(train_X, train_y)

print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)

print(f"\n{preds2}")
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")


create pipeline 2

>>>>>>>init() called.

fit pipeline 2

>>>>>>>fit() called.


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[ 4.21428571 10.3       ]
RMSE: 6.280369834735101e-16

